In [ ]:
from ultralytics import YOLO
from wandb.integration.ultralytics import add_wandb_callback
import os  
import wandb
import warnings 
warnings.filterwarnings(action='ignore', category=UserWarning)

# Load a model
pretrained = "/home/ubuntu/JD-HD/SEGMENT/human_parsing/train9/weights/best.pt"
model = YOLO(pretrained, task="segment"
)  # load a pretrained model (recommended for training)

yaml_file = "configs/fashion_people_detection.yml"
project_name = "human_parsing"

# add_wandb_callback(model)

bs = 72
workers = os.cpu_count()
# Train the model
results = model.train(project=project_name, data=yaml_file, epochs=100, imgsz=1024, batch=bs, workers=workers)

# Finish the W&B run
# wandb.finish()

In [ ]:
import random
from PIL import Image
import requests
from PIL import Image

from utils import *
import yaml

with open(yaml_file, 'r') as file:
    data = yaml.load(file, Loader=yaml.FullLoader)

labels_dict = data.get('names')

In [ ]:
model_path = "/home/ubuntu/JD-HD/SEGMENT/human_parsing/train10/weights/best.pt"
model = YOLO(model_path, task="segment")

In [ ]:
url = "https://www.the-sun.com/wp-content/uploads/sites/6/2023/09/janice-dickinson-attends-36th-carousel-805923729.jpg"
image = Image.open(requests.get(url, stream=True).raw)
image = resize_preserve_aspect_ratio(image, 1024)
results = model(image)

In [ ]:
result = results[0]
labels = result.boxes.cls 
labels = [int(label.item()) for label in labels]
labels = [labels_dict[label] for label in labels]
masks = result.masks.data
masks = [unload_mask(mask) for mask in masks]

masks_dict = dict(zip(labels, masks))
for k, v in masks_dict.items():
    print(k)
    overlay = overlay_mask(image, v)
    display(overlay)